# Overview

For each dataset in the experiment you have below the code for capturing relevant properties of these event logs.

In [4]:
pip install pm4py

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pm4py
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

## BPIC11

In [23]:
bpic11_path = "/content/drive/MyDrive/Tue/Seminars/BPIC11.xes.gz"
bpic11 = pm4py.read_xes(bpic11_path) 

parsing log, completed traces ::   0%|          | 0/1143 [00:00<?, ?it/s]

In [24]:
# Removing columns that less than 1% of non-null values
bpic11 = bpic11.loc[:, bpic11.isnull().sum() <= 0.99*bpic11.shape[0]]
bpic11.head()

,org:group,Number of executions,Specialism code,concept:name,Producer code,Section,Activity code,time:timestamp,lifecycle:transition,case:End date,case:Age,case:Treatment code:2,case:Treatment code:1,case:Diagnosis code:2,case:Specialism code,case:Diagnosis code:1,case:Treatment code,case:Diagnosis:2,case:Diagnosis:1,case:Diagnosis,case:Diagnosis Treatment Combination ID,case:Start date,case:Diagnosis Treatment Combination ID:2,case:Diagnosis code,case:Diagnosis Treatment Combination ID:1,case:concept:name,case:Start date:1,case:End date:1,case:End date:2,case:Specialism code:1,case:Start date:2,case:Specialism code:2,case:Age:1,case:Age:2,case:Treatment code:3,case:Age:3,case:Diagnosis code:3,case:Diagnosis:3,case:Diagnosis Treatment Combination ID:3,case:Specialism code:3,case:Start date:3,case:Treatment code:4,case:Diagnosis code:4,case:Age:4,case:Diagnosis:4,case:Diagnosis Treatment Combination ID:4,case:Specialism code:4,case:End date:3,case:Start date:4,case:Diagnosis code:5,case:Diagnosis:5,case:Treatment code:5,case:Diagnosis Treatment Combination ID:5,case:Specialism code:5,case:Start date:5,case:End date:4,case:End date:5,case:End date:10,case:Start date:10,case:Diagnosis code:10,case:Specialism code:10,case:Diagnosis Treatment Combination ID:10,case:Treatment code:9,case:Treatment code:7,case:Treatment code:8,case:Treatment code:6,case:Treatment code:10,case:Specialism code:6,case:Start date:9,case:Start date:8,case:Start date:7,case:Start date:6,case:Specialism code:9,case:Specialism code:7,case:Specialism code:8,case:Diagnosis:10,case:Diagnosis code:9,case:Diagnosis code:8,case:Diagnosis code:7,case:Diagnosis code:6,case:Diagnosis:8,case:Diagnosis:7,case:Diagnosis:6,case:Diagnosis:9,case:Diagnosis Treatment Combination ID:7,case:Diagnosis Treatment Combination ID:8,case:Diagnosis Treatment Combination ID:9,case:Diagnosis Treatment Combination ID:6,case:End date:8,case:End date:7,case:End date:6
0,Radiotherapy,1,61,1e consult poliklinisch,SRTH,Section 5,410100,2005-01-02 23:00:00+00:00,complete,2006-01-04 22:45:36+00:00,33.0,23.0,13.0,106,7.0,106,103.0,Gynaecologische tumoren,Gynaecologische tumoren,maligniteit cervix,181229.0,2005-01-04 23:14:24+00:00,376908.0,M13,376907.0,00000000,2005-01-02 23:14:24+00:00,2005-01-29 22:45:36+00:00,2005-01-29 22:45:36+00:00,61.0,2005-01-02 23:14:24+00:00,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
1,Radiotherapy,1,61,administratief tarief - eerste pol,SRTH,Section 5,419100,2005-01-02 23:00:00+00:00,complete,2006-01-04 22:45:36+00:00,33.0,23.0,13.0,106,7.0,106,103.0,Gynaecologische tumoren,Gynaecologische tumoren,maligniteit cervix,181229.0,2005-01-04 23:14:24+00:00,376908.0,M13,376907.0,00000000,2005-01-02 23:14:24+00:00,2005-01-29 22:45:36+00:00,2005-01-29 22:45:36+00:00,61.0,2005-01-02 23:14:24+00:00,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
2,Nursing ward,1,7,verlosk.-gynaec. korte kaart kosten-out,SGEH,Section 2,10107,2005-01-04 23:00:00+00:00,complete,2006-01-04 22:45:36+00:00,33.0,23.0,13.0,106,7.0,106,103.0,Gynaecologische tumoren,Gynaecologische tumoren,maligniteit cervix,181229.0,2005-01-04 23:14:24+00:00,376908.0,M13,376907.0,00000000,2005-01-02 23:14:24+00:00,2005-01-29 22:45:36+00:00,2005-01-29 22:45:36+00:00,61.0,2005-01-02 23:14:24+00:00,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3,Obstetrics & Gynaecology clinic,1,7,echografie - genitalia interna,SGEC,Section 2,339486E,2005-01-04 23

In [33]:
num_cases = len(bpic11['case:concept:name'].unique())
num_variants = len(pm4py.get_variants(bpic11, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp'))
num_activities = len(bpic11['concept:name'].unique())
num_events = len(bpic11)
avg_events_per_case = bpic11.groupby('case:concept:name').size().mean()
avg_activities_per_case = bpic11.groupby('case:concept:name')['concept:name'].nunique().mean()

print("Number of cases: {} \nNumber of variants: {} \nNumber of activities: {}".format(num_cases, num_variants, num_activities))
print("Number of events: {} \nAvg of events per case: {} \nAvg of activities per case: {}".format(num_events, avg_events_per_case, avg_activities_per_case))

Number of cases: 1143 
Number of variants: 981 
Number of activities: 624
Number of events: 150291 
Avg of events per case: 131.48818897637796 
Avg of activities per case: 33.201224846894135


In [25]:
case_attrs = [attr for attr in pm4py.get_trace_attributes(bpic11) if attr != 'case:concept:name']
event_attrs = [attr for attr in pm4py.get_event_attributes(bpic11) if attr not in case_attrs + ['case:concept:name']]

num_case_attrs = len(case_attrs)
num_event_attrs = len(event_attrs)

print("Number of event attributes: {} \nNumber of case attributes: {}".format(num_event_attrs, num_case_attrs))

Number of event attributes: 9 
Number of case attributes: 81


In [35]:
# Checking class imbalance
bpic11['case:Label'] = np.where(bpic11['concept:name'].str.contains('spoed'), 1, 0)
bpic11['case:Label'] = bpic11.groupby('case:concept:name')['case:Label'].transform('max')
bpic11.groupby('case:Label')['case:concept:name'].nunique()

case:Label
0    875
1    268
Name: case:concept:name, dtype: int64

## BPIC13

In [11]:
bpic13_path = "/content/drive/MyDrive/Tue/Seminars/BPIC13.xes.gz"
bpic13 = pm4py.read_xes(bpic13_path) 

parsing log, completed traces ::   0%|          | 0/7554 [00:00<?, ?it/s]

In [12]:
# Removing columns that less than 1% of non-null values
bpic13 = bpic13.loc[:, bpic13.isnull().sum() < 0.99*bpic13.shape[0]]
bpic13.head(10)

,org:group,resource country,organization country,org:resource,organization involved,org:role,concept:name,impact,product,lifecycle:transition,time:timestamp,case:concept:name
0,V30,France,fr,Frederic,Org line A2,A2_4,Accepted,Medium,PROD582,In Progress,2010-03-31 14:59:42+00:00,1-364285768
1,V30,France,fr,Frederic,Org line A2,A2_4,Accepted,Medium,PROD582,In Progress,2010-03-31 15:00:56+00:00,1-364285768
2,V5 3rd,France,fr,Frederic,Org line A2,A2_5,Queued,Medium,PROD582,Awaiting Assignment,2010-03-31 15:45:48+00:00,1-364285768
3,V5 3rd,France,fr,Anne Claire,Org line A2,A2_5,Accepted,Medium,PROD582,In Progress,2010-04-06 14:44:07+00:00,1-364285768
4,V30,France,fr,Anne Claire,Org line A2,A2_4,Queued,Medium,PROD582,Awaiting Assignment,2010-04-06 14:44:38+00:00,1-364285768
5,V13 2nd 3rd,France,fr,Anne Claire,Org line A2,A2_5,Accepted,Medium,PROD582,In Progress,2010-04-06 14:44:47+00:00,1-364285768
6,V13 2nd 3rd,France,fr,Anne Claire,Org line A2,A2_5,Completed,Medium,PROD582,Resolved,2010-04-06 14:44:51+00:00,1-364285768
7,V30,France,fr,Anne Claire,Org line A2,A2_4,Queued,Medium,PROD582,Awaiting Assignment,2010-04-06 14:45:07+00:00,1-364285768
8,V30,France,fr,Eric,Org line A2,A2_4,Accepted,Medium,PROD582,In Progress,2010-04-08 10:52:23+00:00,1-364285768
9,V5 3rd,France,fr,Eric,Org line A2,A2_5,Queued,Medium,PROD582,Awaiting Assignment,2010-04-08 10:53:35+00:00,1-364285768


In [38]:
num_cases = len(bpic13['case:concept:name'].unique())
num_variants = len(pm4py.get_variants(bpic13, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp'))
num_activities = len(bpic13['concept:name'].unique())
num_events = len(bpic13)
avg_events_per_case = bpic13.groupby('case:concept:name').size().mean()
avg_activities_per_case = bpic13.groupby('case:concept:name')['concept:name'].nunique().mean()

num_event_attrs = len(pm4py.get_event_attributes(bpic13))
num_case_attrs = len(pm4py.get_trace_attributes(bpic13))

print("Number of cases: {} \nNumber of variants: {} \nNumber of activities: {}".format(num_cases, num_variants, num_activities))
print("Number of events: {} \nAvg of events per case: {} \nAvg of activities per case: {}".format(num_events, avg_events_per_case, avg_activities_per_case))
print("Number of event attributes: {} \nNumber of case attributes: {}".format(num_event_attrs, num_case_attrs))

Number of cases: 7554 
Number of variants: 1511 
Number of activities: 4
Number of events: 65533 
Avg of events per case: 8.675271379401641 
Avg of activities per case: 2.5965051628276408
Number of event attributes: 12 
Number of case attributes: 1


In [39]:
case_attrs = [attr for attr in pm4py.get_trace_attributes(bpic13) if attr != 'case:concept:name']
event_attrs = [attr for attr in pm4py.get_event_attributes(bpic13) if attr not in case_attrs + ['case:concept:name']]

num_case_attrs = len(case_attrs)
num_event_attrs = len(event_attrs)

print("Number of event attributes: {} \nNumber of case attributes: {}".format(num_event_attrs, num_case_attrs))

Number of event attributes: 11 
Number of case attributes: 0


In [40]:
# Checking class imbalance
bpic13['case:Label'] = np.where(bpic13['org:group'].str.contains('2nd') | bpic13['org:group'].str.contains('3rd'), 1, 0)
bpic13['case:Label'] = bpic13.groupby('case:concept:name')['case:Label'].transform('max')
bpic13.groupby('case:Label')['case:concept:name'].nunique()

case:Label
0    4546
1    3008
Name: case:concept:name, dtype: int64

## RL

In [9]:
rl_path = "/content/drive/MyDrive/Tue/Seminars/RL.xes.gz"
rl = pm4py.read_xes(rl_path) 

parsing log, completed traces ::   0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
# Removing columns that less than 1% of non-null values
rl = rl.loc[:, rl.isnull().sum() < 0.99*rl.shape[0]]
rl.head(10)

,org:resource,time:timestamp,concept:name,lifecycle:transition,case:concept:name,case:description,result
0,Mike,2005-12-31 23:00:00+00:00,invite reviewers,start,1,Simulated process instance,NaN
1,Mike,2006-01-04 23:00:00+00:00,invite reviewers,complete,1,Simulated process instance,NaN
2,Mary,2006-01-04 23:00:00+00:00,get review 3,complete,1,Simulated process instance,reject
3,Pam,2006-01-05 23:00:00+00:00,get review 1,complete,1,Simulated process instance,reject
4,__INVALID__,2006-01-14 23:00:00+00:00,time-out 2,complete,1,Simulated process instance,NaN
5,Anne,2006-01-15 23:00:00+00:00,collect reviews,start,1,Simulated process instance,NaN
6,Anne,2006-01-15 23:00:00+00:00,collect reviews,complete,1,Simulated process instance,NaN
7,Wil,2006-01-15 23:00:00+00:00,decide,start,1,Simulated process instance,NaN
8,Wil,2006-01-16 23:00:00+00:00,decide,complete,1,Simulated process instance,NaN
9,Mike,2006-01-16 23:00:00+00:00,invite additional reviewer,start,1,Simulated process instance,NaN


In [43]:
num_cases = len(rl['case:concept:name'].unique())
num_variants = len(pm4py.get_variants(rl, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp'))
num_activities = len(rl['concept:name'].unique())
num_events = len(rl)
avg_events_per_case = rl.groupby('case:concept:name').size().mean()
avg_activities_per_case = rl.groupby('case:concept:name')['concept:name'].nunique().mean()

print("Number of cases: {} \nNumber of variants: {} \nNumber of activities: {}".format(num_cases, num_variants, num_activities))
print("Number of events: {} \nAvg of events per case: {} \nAvg of activities per case: {}".format(num_events, avg_events_per_case, avg_activities_per_case))

Number of cases: 10000 
Number of variants: 4118 
Number of activities: 14
Number of events: 236360 
Avg of events per case: 23.636 
Avg of activities per case: 9.5632


In [44]:
case_attrs = [attr for attr in pm4py.get_trace_attributes(rl) if attr != 'case:concept:name']
event_attrs = [attr for attr in pm4py.get_event_attributes(rl) if attr not in case_attrs + ['case:concept:name']]

num_case_attrs = len(case_attrs)
num_event_attrs = len(event_attrs)

print("Number of event attributes: {} \nNumber of case attributes: {}".format(num_event_attrs, num_case_attrs))

Number of event attributes: 5 
Number of case attributes: 1


In [46]:
# Checking class imbalance
rl['case:Label'] = np.where(rl['concept:name'] == 'reject', 1, 0)
rl['case:Label'] = rl.groupby('case:concept:name')['case:Label'].transform('max')
rl.groupby('case:Label')['case:concept:name'].nunique()

case:Label
0    5068
1    4932
Name: case:concept:name, dtype: int64

## RTFM

In [7]:
rtfm_path = "/content/drive/MyDrive/Tue/Seminars/RTFM.xes.gz"
rtfm = pm4py.read_xes(rtfm_path) 

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

In [8]:
# Removing columns that less than 1% of non-null values
rtfm = rtfm.loc[:, rtfm.isnull().sum() < 0.99*rtfm.shape[0]]
rtfm.head(10)

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-23 22:00:00+00:00,157.0,0.0,A1,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-04 23:00:00+00:00,NaN,NaN,A1,11.0,NaN,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.0,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN
5,71.5,NaN,NaN,Add penalty,NaN,NaN,complete,2007-03-15 23:00:00+00:00,NaN,NaN,A100,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,Send for Credit Collection,NaN,NaN,complete,2009-03-29 22:00:00+00:00,NaN,NaN,A100,NaN,NaN,NaN,NaN
7,36.0,561,NIL,Create Fine,A,0.0,complete,2007-03-08 23:00:00+00:00,157.0,0.0,A10000,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2007-07-16 22:00:00+00:00,NaN,NaN,A10000,13.0,NaN,NaN,NaN
9,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-08-01 22:00:00+00:00,NaN,NaN,A10000,NaN,P,P,NaN


In [49]:
num_cases = len(rtfm['case:concept:name'].unique())
num_variants = len(pm4py.get_variants(rtfm, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp'))
num_activities = len(rtfm['concept:name'].unique())
num_events = len(rtfm)
avg_events_per_case = rtfm.groupby('case:concept:name').size().mean()
avg_activities_per_case = rtfm.groupby('case:concept:name')['concept:name'].nunique().mean()

num_event_attrs = len(pm4py.get_event_attributes(rtfm))
num_case_attrs = len(pm4py.get_trace_attributes(rtfm))

print("Number of cases: {} \nNumber of variants: {} \nNumber of activities: {}".format(num_cases, num_variants, num_activities))
print("Number of events: {} \nAvg of events per case: {} \nAvg of activities per case: {}".format(num_events, avg_events_per_case, avg_activities_per_case))
print("Number of event attributes: {} \nNumber of case attributes: {}".format(num_event_attrs, num_case_attrs))

Number of cases: 150370 
Number of variants: 231 
Number of activities: 11
Number of events: 561470 
Avg of events per case: 3.7339229899581032 
Avg of activities per case: 3.6814790184212276
Number of event attributes: 15 
Number of case attributes: 1


In [50]:
case_attrs = [attr for attr in pm4py.get_trace_attributes(rtfm) if attr != 'case:concept:name']
event_attrs = [attr for attr in pm4py.get_event_attributes(rtfm) if attr not in case_attrs + ['case:concept:name']]

num_case_attrs = len(case_attrs)
num_event_attrs = len(event_attrs)

print("Number of event attributes: {} \nNumber of case attributes: {}".format(num_event_attrs, num_case_attrs))

Number of event attributes: 14 
Number of case attributes: 0


In [51]:
# Checking class imbalance
rtfm['case:Label'] = np.where(rtfm['concept:name'] == 'Appeal to Judge', 1, 0)
rtfm['case:Label'] = rtfm.groupby('case:concept:name')['case:Label'].transform('max')
rtfm.groupby('case:Label')['case:concept:name'].nunique()

case:Label
0    149815
1       555
Name: case:concept:name, dtype: int64